In [1]:
import environs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common import exceptions
from selenium.webdriver.common.action_chains import ActionChains
from datetime import date
import time
from pprint import pprint
import pandas as pd

In [2]:
goods_category = input('Укажитенаименование товара:\n')
driver = webdriver.Chrome(executable_path='./chromedriver')
driver.implicitly_wait(10)
driver.fullscreen_window()
driver.get('https://www.wildberries.ru/')

Укажитенаименование товара:
кроссовки мужские


In [3]:
elem = driver.find_element(By.CLASS_NAME, 'nav-element__search')
elem.click()

In [4]:
elem = driver.find_element(By.ID, 'mobileSearchInput')
elem.send_keys(goods_category)

In [5]:
elem.send_keys(Keys.ENTER)

In [6]:
goods = driver.find_elements(By.CLASS_NAME, 'product-card')

In [7]:
goods_list = []
for i in range(len(goods)):
    if i > 0:
        goods = driver.find_elements(By.CLASS_NAME, 'product-card')
    try:
        goods[i].click()
        time.sleep(3)
        goods_data = {}
        brand = driver.find_elements(By.XPATH, '//span[@data-link="text{:product^brandName}"]')[1].text
        name = driver.find_elements(By.XPATH, '//span[@data-link="text{:product^goodsName}"]')[1].text
        price = driver.find_element(By.CLASS_NAME, 'price-block__price-wrap').text
        price = price.split('\n')
        seller = driver.find_element(By.CLASS_NAME, 'seller__name').text
        number_sales = driver.find_elements(By.XPATH, '//span[contains(@data-link, "productCardOrderCount")]')[1].text
        goods_data['Наименование товара'] = name
        goods_data['Брэнд'] = brand
        try:
            goods_data['Цена со скидкой'] = price[0]
        except IndexError:
            goods_data['Цена со скидкой'] = None
        try:
            goods_data['Цена без скидки'] = price[1]
        except IndexError:
            goods_data['Цена без скидки'] = price[0]
        goods_data['Продавец'] = seller
        goods_data['Количество продаж'] = number_sales
        goods_list.append(goods_data)
        next_good = driver.find_element(By.CLASS_NAME, 'sw-slider-kt-mix__btn')
        next_good.click()
    except IndexError:
        break
    time.sleep(3)

In [8]:
len(goods_list)

105

In [9]:
df = pd.DataFrame(goods_list)
df.head()

,Наименование товара,Брэнд,Цена со скидкой,Цена без скидки,Продавец,Количество продаж
0,Кроссовки,ICON-TRADE,2 475 ₽,3 536 ₽,ИП Гончаров Дмитрий Владимирович,менее 5 раз
1,Кроссовки зимние/Кроссовки мужские/Кроссовки с...,SJ STYLE,2 980 ₽,5 400 ₽,ИП Супиев Айгун Низамиевич,более 10 раз
2,Кроссовки HOOPS 2.0 MID,adidas,4 412 ₽,6 999 ₽,ООО ВАЙЛДБЕРРИЗ,более 2 300 раз
3,Кроссовки HOOPS 2.0 MID,adidas,4 412 ₽,6 999 ₽,ООО ВАЙЛДБЕРРИЗ,более 8 800 раз
4,Кроссовки HOOPS 2.0 MID CARBON/CBLACK/WHITIN,adidas,4 412 ₽,6 999 ₽,ООО ВАЙЛДБЕРРИЗ,более 2 200 раз


In [10]:
df.to_excel('men_running_shoes.xlsx')